## 🎉 Master-Worker Chatbot Ready!

Your conversational loan chatbot with **Master-Worker Architecture** is fully functional!

### Architecture:
✅ **Master Agent** - Main orchestrator that:
  - Receives all user messages
  - Analyzes intent
  - Delegates to appropriate Worker Agents
  - Maintains conversation control

✅ **Worker Agents** - Specialized handlers:
  - **Information Worker** - Provides loan details
  - **Loan Application Worker** - Manages full pipeline with sub-agents:
    - Sanction Sub-Agent
    - Sales Sub-Agent (home loans)
    - Verification Sub-Agent
    - Underwriting Sub-Agent
    - Final Sanction Sub-Agent

✅ **Control Flow:**
  1. User → Master Agent
  2. Master → Worker Agent (with state)
  3. Worker processes request
  4. Worker → Returns to Master (or END)
  5. Master continues conversation

### Key Features:
- Master maintains conversation context
- Workers are task-specific and autonomous
- Clean separation of concerns
- Easy to add new worker agents

### Example Queries:
- "What are education loan interest rates?" → Info Worker
- "I want to apply for a home loan" → Loan Worker Pipeline
- "Hello!" → Master handles directly

In [ ]:
def start_interactive_chat():
    """Start an interactive chat session with the loan chatbot"""
    print("\n" + "=" * 60)
    print("🏦 LOAN CHATBOT - Interactive Mode")
    print("=" * 60)
    print("Welcome! I can help you with:")
    print("  • Loan information (education, home, personal, car loans)")
    print("  • Loan applications")
    print("  • General questions")
    print("\nType 'exit', 'quit', or 'bye' to end the conversation")
    print("Type 'reset' to start a new conversation")
    print("=" * 60)
    
    chatbot.reset_conversation()
    
    while True:
        try:
            user_input = input("\n💬 You: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("\n👋 Thank you for using Loan Chatbot! Have a great day!")
                break
            
            if user_input.lower() == 'reset':
                chatbot.reset_conversation()
                print("\n🔄 Conversation reset. Starting fresh!")
                continue
            
            # Get response
            response = chatbot.chat(user_input)
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again or type 'exit' to quit.")

# Uncomment the line below to start interactive chat
# start_interactive_chat()

print("✅ Interactive chat function ready!")
print("Run: start_interactive_chat() to begin chatting")

## Step 21: Interactive Chat Loop

Start an interactive conversation with the chatbot

In [ ]:
# Test 3: Loan application
print("\n" + "=" * 60)
print("TEST 3: Loan Application (Home Loan)")
print("=" * 60)

chatbot.reset_conversation()
response = chatbot.chat("I want to apply for a home loan of $300,000")

## Step 20: Test the Chatbot - Loan Application Flow

Test the complete loan application process

In [ ]:
# Test 2: General conversation
print("\n" + "=" * 60)
print("TEST 2: General Conversation")
print("=" * 60)

chatbot.reset_conversation()
response = chatbot.chat("Hello! How are you today?")

## Step 19: Test the Chatbot - General Conversation

Test casual conversation

In [ ]:
# Test 1: Ask for information
print("=" * 60)
print("TEST 1: Information Request")
print("=" * 60)

chatbot.reset_conversation()
response = chatbot.chat("What are the interest rates for education loans?")

## Step 18: Test the Chatbot - Information Request

Test asking for loan information

In [ ]:
class LoanChatbot:
    """Conversational chatbot with Master-Worker Agent Architecture"""
    
    def __init__(self):
        self.reset_conversation()
    
    def reset_conversation(self):
        """Start a new conversation"""
        self.state = {
            "messages": [],
            "question_type": "",
            "loan_type": "",
            "loan_application": {},
            "current_agent": "",
            "worker_agent": "",
            "worker_response": "",
            "needs_clarification": False,
            "verification_status": "pending",
            "conversation_active": True,
            "delegate_to_worker": False,
            "return_to_master": False
        }
    
    def chat(self, user_message: str) -> str:
        """Send a message and get response"""
        print(f"\n👤 You: {user_message}")
        print("─" * 60)
        
        # Add user message
        self.state["messages"].append(HumanMessage(content=user_message))
        
        # Run the graph (Master orchestrates everything)
        self.state = app.invoke(self.state)
        
        # Get the last AI message (filter out system messages)
        ai_messages = [m for m in self.state["messages"] if isinstance(m, AIMessage)]
        if ai_messages:
            response = ai_messages[-1].content
            print(f"\n🤖 Assistant: {response}")
            return response
        
        return "I'm sorry, I couldn't process that. Can you try again?"
    
    def get_conversation_history(self):
        """Return full conversation"""
        return self.state["messages"]
    
    def show_agent_flow(self):
        """Show which agents handled the conversation"""
        print("\n📋 Agent Flow:")
        for msg in self.state["messages"]:
            if isinstance(msg, SystemMessage) and ("Master Agent Note" in msg.content or "Worker" in msg.content):
                print(f"   • {msg.content[:80]}...")

# Initialize chatbot
chatbot = LoanChatbot()
print("✅ Chatbot with Master-Worker architecture ready!")

## Step 17: Create Chat Interface Functions

Helper functions to interact with the chatbot

In [ ]:
from IPython.display import Image, display

try:
    # Try to visualize the graph
    graph_image = app.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    print("✅ Graph visualization displayed!")
except Exception as e:
    print(f"⚠️ Could not visualize graph: {e}")
    print("Install graphviz for visualization: pip install pygraphviz")

## Step 16: Visualize the Graph

Display the workflow structure (requires graphviz)

In [ ]:
# Create the StateGraph with Master-Worker Architecture
workflow = StateGraph(AgentState)

# Add Master Agent
workflow.add_node("master", master_agent)

# Add Worker Agents
workflow.add_node("info_worker", information_worker)
workflow.add_node("loan_worker", loan_application_worker)

# Add Loan Application Sub-Agents
workflow.add_node("sanction_sub", sanction_sub_agent)
workflow.add_node("sales_sub", sales_sub_agent)
workflow.add_node("verification_sub", verification_sub_agent)
workflow.add_node("underwriting_sub", underwriting_sub_agent)
workflow.add_node("final_sanction_sub", final_sanction_sub_agent)

# Set Master as entry point
workflow.set_entry_point("master")

# Master delegates to workers
workflow.add_conditional_edges(
    "master",
    route_from_master,
    {
        "info_worker": "info_worker",
        "loan_worker": "loan_worker",
        "end": END
    }
)

# Info Worker returns to Master or ends
workflow.add_edge("info_worker", END)

# Loan Worker starts the pipeline with sanction
workflow.add_edge("loan_worker", "sanction_sub")

# Sanction routes to Sales (home) or Verification (other)
workflow.add_conditional_edges(
    "sanction_sub",
    route_loan_pipeline,
    {
        "sales_sub": "sales_sub",
        "verification_sub": "verification_sub"
    }
)

# Sales goes to Verification
workflow.add_edge("sales_sub", "verification_sub")

# Verification goes to Underwriting
workflow.add_edge("verification_sub", "underwriting_sub")

# Underwriting goes to Final Sanction
workflow.add_edge("underwriting_sub", "final_sanction_sub")

# Final Sanction completes and returns to Master
workflow.add_edge("final_sanction_sub", END)

# Compile the graph
app = workflow.compile()

print("✅ Master-Worker LangGraph compiled successfully!")
print("\n📊 Architecture:")
print("   ┌─────────────────┐")
print("   │  MASTER AGENT   │ ← Entry Point")
print("   └────────┬────────┘")
print("            │")
print("     ┌──────┴──────┐")
print("     │             │")
print("┌────▼────┐   ┌────▼────────────────┐")
print("│  INFO   │   │  LOAN APPLICATION   │")
print("│ WORKER  │   │     WORKER          │")
print("└────┬────┘   └────┬────────────────┘")
print("     │             │")
print("     │        ┌────┼────┬─────┬──────┐")
print("     │        │    │    │     │      │")
print("     │     Sanction│ Sales│ Verify│ Under│ Final")
print("     │             │    │     │      │")
print("     └─────────────┴────┴─────┴──────┘")
print("               │")
print("          Return to MASTER or END")

## Step 15: Build the LangGraph Workflow

Create the complete graph with all nodes and edges

In [ ]:
def route_from_master(state: AgentState) -> Literal["info_worker", "loan_worker", "end"]:
    """Route from Master Agent to appropriate Worker"""
    if state.get("delegate_to_worker", False):
        worker = state.get("worker_agent", "")
        print(f"   → Routing to: {worker}")
        if worker == "info_worker":
            return "info_worker"
        elif worker == "loan_worker":
            return "loan_worker"
    return "end"

def route_loan_pipeline(state: AgentState) -> Literal["sales_sub", "verification_sub"]:
    """Route within loan application pipeline"""
    # If home loan, go to sales agent first
    if state.get("loan_type") == "home loan":
        return "sales_sub"
    # Otherwise, go to verification
    return "verification_sub"

def check_worker_completion(state: AgentState) -> Literal["master", "end"]:
    """Check if worker is done and should return to master"""
    if state.get("return_to_master", False):
        print("   ← Returning to Master Agent")
        state["return_to_master"] = False
        state["delegate_to_worker"] = False
        return "master"
    return "end"

print("✅ Routing logic defined for Master-Worker architecture!")

## Step 14: Define Routing Logic

Router functions determine the next node based on state

In [ ]:
def final_sanction_sub_agent(state: AgentState) -> AgentState:
    """
    Final Sanction Sub-Agent: Part of Loan Application Worker
    Makes final decision and returns control to Master Agent
    """
    print("   🔹 Final Sanction Sub-Agent: Making final decision...")
    state["current_agent"] = "final_sanction"
    
    system_prompt = """You are the Final Sanction Sub-Agent (part of the Loan Application Worker team). 
    Make a final decision:
    1. APPROVE / CONDITIONAL APPROVAL / REJECT
    2. Clearly explain the reasoning
    3. Provide specific next steps
    4. Include processing timeline
    
    Be professional and supportive."""
    
    clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
    messages = [SystemMessage(content=system_prompt)] + clean_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    state["worker_response"] = response.content
    state["verification_status"] = "completed"
    state["return_to_master"] = True
    
    # Add a system message marking completion
    completion_msg = """Loan Application Worker: Pipeline complete. 
    Final decision communicated. Returning control to Master Agent."""
    state["messages"].append(SystemMessage(content=completion_msg))
    
    print("   ✓ Pipeline complete, returning to Master")
    return state

print("✅ Final Sanction Sub-Agent created!")

## Step 13: Final Sanction Agent

Makes final approval/rejection decision and provides next steps

In [ ]:
def underwriting_sub_agent(state: AgentState) -> AgentState:
    """
    Underwriting Sub-Agent: Part of Loan Application Worker
    Assesses risk and determines loan terms
    """
    print("   🔹 Underwriting Sub-Agent: Assessing application...")
    state["current_agent"] = "underwriting"
    
    system_prompt = """You are an Underwriting Sub-Agent (part of the Loan Application Worker team). 
    Analyze the application and provide:
    1. Risk assessment (Low/Medium/High)
    2. Recommended loan amount
    3. Suggested interest rate range
    4. Any special conditions
    
    Be analytical and fair."""
    
    clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
    messages = [SystemMessage(content=system_prompt)] + clean_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    return state

print("✅ Underwriting Sub-Agent created!")

## Step 12: Underwriting Agent

Assesses risk and determines loan terms

In [ ]:
def verification_sub_agent(state: AgentState) -> AgentState:
    """
    Verification Sub-Agent: Part of Loan Application Worker
    Reviews and confirms application details
    """
    print("   🔹 Verification Sub-Agent: Reviewing details...")
    state["current_agent"] = "verification"
    
    system_prompt = """You are a Verification Sub-Agent (part of the Loan Application Worker team). 
    Your role is to:
    1. Summarize all information collected
    2. Ask the applicant to confirm correctness
    3. List required documents
    4. Explain next steps
    
    Be thorough and professional."""
    
    clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
    messages = [SystemMessage(content=system_prompt)] + clean_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    state["verification_status"] = "pending"
    return state

print("✅ Verification Sub-Agent created!")

## Step 11: Verification Agent

Reviews collected information and asks for confirmation

In [ ]:
def sales_sub_agent(state: AgentState) -> AgentState:
    """
    Sales Sub-Agent: Part of Loan Application Worker (Home Loan Specialist)
    """
    print("   🔹 Sales Sub-Agent: Gathering home loan details...")
    state["current_agent"] = "sales"
    
    system_prompt = """You are a Sales Sub-Agent (part of the Loan Application Worker team) 
    specializing in home loans. Ask detailed questions about:
    1. Property details (value, location, type)
    2. Down payment availability
    3. Employment status and monthly income
    4. Credit history
    
    Ask questions naturally, one at a time."""
    
    clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
    messages = [SystemMessage(content=system_prompt)] + clean_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    return state

print("✅ Sales Sub-Agent created!")

## Step 10: Sales Agent (Home Loan Specialist)

Specialized agent for home loans with detailed property questions

In [ ]:
def sanction_sub_agent(state: AgentState) -> AgentState:
    """
    Sanction Sub-Agent: Part of Loan Application Worker
    Gathers initial loan application information
    """
    print("   🔹 Sanction Sub-Agent: Gathering information...")
    state["current_agent"] = "sanction"
    
    # Get user messages only
    user_messages = [m for m in state["messages"] if isinstance(m, HumanMessage)]
    last_user_message = user_messages[-1].content.lower() if user_messages else ""
    
    # Detect loan type from message
    for loan_type in LOAN_INFORMATION_DB.keys():
        if loan_type.replace(" ", "") in last_user_message.replace(" ", ""):
            state["loan_type"] = loan_type
            break
    
    system_prompt = """You are a Sanction Sub-Agent (part of the Loan Application Worker team). 
    Your role is to:
    1. Warmly welcome the applicant
    2. Ask about the type of loan they need (if not mentioned)
    3. Ask about the loan amount required
    4. Ask about the purpose of the loan
    5. Keep questions clear and ask ONE question at a time
    
    Be professional, friendly, and helpful."""
    
    clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
    messages = [SystemMessage(content=system_prompt)] + clean_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    state["needs_clarification"] = True
    return state

print("✅ Sanction Sub-Agent created!")

## Step 9: Sanction Agent (Initial Handler)

First agent in the loan application pipeline - gathers basic information

In [ ]:
def loan_application_worker(state: AgentState) -> AgentState:
    """
    Loan Application Worker: Manages the entire loan application pipeline
    - Coordinates sub-agents (Sanction, Sales, Verification, Underwriting, Final)
    - Handles the multi-step loan application process
    - Returns to Master Agent when complete
    """
    print("👷 Loan Application Worker: Starting pipeline...")
    state["current_agent"] = "loan_worker"
    
    # This worker orchestrates the loan application sub-workflow
    # For now, starting with sanction agent
    state["delegate_to_worker"] = False
    return state

print("✅ Loan Application Worker created!")

## Step 8: Loan Application Worker Agent

Worker agent that manages the complete loan application pipeline

In [ ]:
def information_worker(state: AgentState) -> AgentState:
    """
    Information Worker Agent: Handles information requests
    - Searches synthetic cache for loan details
    - Uses LLM for complex queries
    - Returns control to Master Agent after responding
    """
    print("👷 Information Worker: Processing request...")
    state["current_agent"] = "info_worker"
    
    # Get the actual user question (filter out system messages)
    user_messages = [m for m in state["messages"] if isinstance(m, HumanMessage)]
    last_user_message = user_messages[-1].content.lower() if user_messages else ""
    
    # Search in synthetic cache
    response = ""
    found_in_cache = False
    
    for loan_type, details in LOAN_INFORMATION_DB.items():
        if loan_type.replace(" ", "") in last_user_message.replace(" ", ""):
            found_in_cache = True
            response = f"📋 **{loan_type.title()} Information:**\n\n"
            
            for key, value in details.items():
                if key == "documents":
                    response += f"**{key.replace('_', ' ').title()}:**\n"
                    for doc in value:
                        response += f"  • {doc}\n"
                else:
                    response += f"**{key.replace('_', ' ').title()}:** {value}\n"
            
            response += "\n💡 Would you like to apply for this loan or need more information?"
            break
    
    # If not found in cache, use LLM
    if not found_in_cache:
        system_prompt = """You are an Information Worker Agent specializing in loan information. 
        Provide accurate, concise information about loans. 
        Be friendly and professional."""
        
        clean_messages = [m for m in state["messages"] if not (isinstance(m, SystemMessage) and "Master Agent Note" in m.content)]
        messages = [SystemMessage(content=system_prompt)] + clean_messages
        ai_response = llm.invoke(messages)
        response = ai_response.content
    
    state["messages"].append(AIMessage(content=response))
    state["worker_response"] = response
    state["return_to_master"] = True
    state["conversation_active"] = True
    
    print("   ✓ Information provided, returning to Master")
    return state

print("✅ Information Worker created!")

## Step 7: Information Worker Agent

Worker agent that handles information requests and returns to Master

In [ ]:
def master_agent(state: AgentState) -> AgentState:
    """
    Master Agent: Main orchestrator that routes to worker agents
    - Analyzes user intent
    - Decides which worker agent should handle the request
    - Delegates work to appropriate worker
    - Receives results back and responds to user
    """
    last_message = state["messages"][-1].content.lower()
    
    state["current_agent"] = "master"
    print("🎯 Master Agent: Analyzing request...")
    
    # Check for loan application intent
    application_keywords = ["apply", "application", "want loan", "need loan", "get loan", "take loan"]
    if any(keyword in last_message for keyword in application_keywords):
        state["question_type"] = "loan_application"
        state["worker_agent"] = "loan_worker"
        state["delegate_to_worker"] = True
        print("   ↳ Delegating to: Loan Application Worker")
        
        # Master prepares context for worker
        delegation_msg = f"""Master Agent Note: User wants to apply for a loan. 
        Delegating to Loan Application Worker to handle the full pipeline."""
        state["messages"].append(SystemMessage(content=delegation_msg))
        return state
    
    # Check for information request
    info_keywords = ["interest rate", "eligibility", "tenure", "how much", "details", 
                     "information", "tell me about", "what is", "requirements", "documents"]
    if any(keyword in last_message for keyword in info_keywords):
        state["question_type"] = "information"
        state["worker_agent"] = "info_worker"
        state["delegate_to_worker"] = True
        print("   ↳ Delegating to: Information Worker")
        
        delegation_msg = f"""Master Agent Note: User needs loan information. 
        Delegating to Information Worker."""
        state["messages"].append(SystemMessage(content=delegation_msg))
        return state
    
    # General conversation - handled by master itself
    state["question_type"] = "general"
    state["delegate_to_worker"] = False
    print("   ↳ Handling directly: General conversation")
    
    system_prompt = """You are the Master Agent - a friendly loan chatbot coordinator. 
    Handle general conversation warmly and professionally. 
    Guide users toward specific services when appropriate.
    Keep responses concise but helpful."""
    
    # Filter out system delegation messages for the LLM
    user_messages = [m for m in state["messages"] if not isinstance(m, SystemMessage) or "Master Agent Note" not in m.content]
    messages = [SystemMessage(content=system_prompt)] + user_messages
    response = llm.invoke(messages)
    
    state["messages"].append(AIMessage(content=response.content))
    state["conversation_active"] = True
    return state

print("✅ Master Agent created!")

## Step 6: Master Agent Node

The Master Agent is the main orchestrator that:
1. Receives all user messages
2. Analyzes intent and decides which worker to delegate to
3. Passes state to worker agents
4. Receives results and controls the conversation flow

In [ ]:
# Synthetic Cache - Simulates database of loan information
LOAN_INFORMATION_DB = {
    "education loan": {
        "interest_rate": "6.5% - 9.5% per annum",
        "max_amount": "Up to $100,000",
        "tenure": "5-15 years",
        "processing_fee": "1% of loan amount",
        "eligibility": "Students admitted to recognized institutions",
        "documents": ["Admission letter", "Fee structure", "ID proof", "Income proof of co-applicant"]
    },
    "home loan": {
        "interest_rate": "7.0% - 10.0% per annum",
        "max_amount": "Up to $500,000 (depends on property value)",
        "tenure": "5-30 years",
        "processing_fee": "0.5% - 1% of loan amount",
        "eligibility": "Salaried/Self-employed with stable income, age 21-65",
        "documents": ["Property documents", "Income proof", "ID proof", "Bank statements"]
    },
    "personal loan": {
        "interest_rate": "10.5% - 24.0% per annum",
        "max_amount": "Up to $50,000",
        "tenure": "1-5 years",
        "processing_fee": "2% of loan amount",
        "eligibility": "Individuals with credit score > 700",
        "documents": ["ID proof", "Address proof", "Income proof", "Bank statements"]
    },
    "car loan": {
        "interest_rate": "8.0% - 12.0% per annum",
        "max_amount": "Up to 90% of vehicle cost",
        "tenure": "1-7 years",
        "processing_fee": "1% of loan amount",
        "eligibility": "Salaried/Self-employed individuals",
        "documents": ["Vehicle quotation", "Income proof", "ID proof", "Bank statements"]
    }
}

print("✅ Loan information database loaded!")
print(f"Available loan types: {list(LOAN_INFORMATION_DB.keys())}")

## Step 5: Create Synthetic Cache for Loan Information

This simulates a knowledge base with common loan information

In [ ]:
# Choose your LLM provider (uncomment one)

# Option 1: OpenAI
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    print("✅ Using OpenAI GPT-4")
except:
    llm = None

# Option 2: Anthropic Claude (uncomment if using)
# try:
#     llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0.7)
#     print("✅ Using Anthropic Claude")
# except:
#     llm = None

# Option 3: Google Gemini (uncomment if using)
# try:
#     llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
#     print("✅ Using Google Gemini")
# except:
#     llm = None

if llm is None:
    print("⚠️ No LLM configured. Please set API keys in .env file")

## Step 4: Initialize LLM

You can use OpenAI, Anthropic, or Google Gemini. Configure your API key in a `.env` file.

In [ ]:
class AgentState(TypedDict):
    """State schema for the chatbot with Master-Worker architecture"""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    question_type: str  # "information", "general", "loan_application"
    loan_type: str  # "education", "home", "personal", etc.
    loan_application: dict  # Stores application data
    current_agent: str  # Tracks which agent is active
    worker_agent: str  # Which worker is handling this
    worker_response: str  # Response from worker agent
    needs_clarification: bool  # Whether more info is needed
    verification_status: str  # "pending", "approved", "rejected"
    conversation_active: bool  # Keep conversation going
    delegate_to_worker: bool  # Flag to delegate to worker
    return_to_master: bool  # Flag to return control to master

print("✅ State schema defined with Master-Worker architecture!")

## Step 3: Define State Schema

The state maintains conversation history, routing information, and application data

In [ ]:
from typing import TypedDict, Annotated, Sequence, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import operator
from dotenv import load_dotenv
import os
import json

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")

## Step 2: Import Required Libraries

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai langchain-anthropic langchain-google-genai python-dotenv

## Step 1: Install Required Packages

# Loan Chatbot Master Agent with LangGraph

This notebook implements a conversational loan chatbot using LangGraph that follows a multi-agent workflow for handling:
- Information requests about loans
- General conversation
- Loan applications with full processing pipeline